In [1]:
import numpy as np
import sunode
import sunode.wrappers.as_pytensor
import pymc as pm
import pandas as pd
from sirpy.models.SIR import SIR
import arviz as az
import jax


In [2]:
data = pd.read_csv("covid_japan_processed.csv", parse_dates=[0], index_col=0)
t = (data.index - data.index.min()) / (data.index.max() - data.index.min())
data.reset_index(drop=True, inplace=True)
train = data.iloc[0:100,:]
test = data.iloc[100::,:]
print(f"Train is the first {len(train)} days and test is remaining {len(test)} days.") #Typo

Train is the first 100 days and test is remaining 23 days.


In [3]:
def sir(t, y, p):
    """Right hand side of Lotka-Volterra equation.

    All inputs are dataclasses of sympy variables, or in the case
    of non-scalar variables numpy arrays of sympy variables.
    """
    return {
        'S': -p.beta * y.S * y.I,
        'I': p.beta * y.S * y.I - p.gamma * y.I,
        'R': p.gamma * y.I
    }


In [4]:
times = t.to_numpy()

In [5]:
sir_m = pm.Model()
with sir_m:
    # Define initial conditions
    S0 = pm.Uniform('S0', lower=0, upper=1)
    I0 = pm.Uniform('I0', lower=0, upper=1-S0)
    R0 = pm.Uniform('R0', lower=0, upper=1-I0-S0)

    # Define priors
    beta = pm.Normal("beta", 0, 1)
    gamma = pm.Normal("gamma", 0, 1)
    sd = pm.HalfNormal('sd')

    # Solve the ODE
    y_hat, _, problem, solver, _, _ = sunode.wrappers.as_pytensor.solve_ivp(
        y0={
	    # The initial conditions of the ode. Each variable
	    # needs to specify a PyTensor or numpy variable and a shape.
	    # This dict can be nested.
            'S': (S0, ()),
            'I': (I0, ()),
            'R': (R0, ()),
        },
        params={
	    # Each parameter of the ode. sunode will only compute derivatives
	    # with respect to PyTensor variables. The shape needs to be specified
	    # as well. It it infered automatically for numpy variables.
	    # This dict can be nested.
            'beta': (beta, ()),
            'gamma': (gamma, ()),
            '_dummy': (np.array(1.), ()),
        },
	# A functions that computes the right-hand-side of the ode using
	# sympy variables.
        rhs=sir,
	# The time points where we want to access the solution
        tvals=times,
        t0=times[0],
    )
    # We can access the individual variables of the solution using the
    # variable names.

    # likehoods
    # posterior prop prior x likehood
    # prior x likehood --> MCMC --> frecuencias relativas
    # Si suficiente tiempo la cadena de markov converge y representa las verdaderas frecuencais
    pm.Normal('I', mu=y_hat["I"], sigma=sd, observed=data.iloc[:,1])
    pm.Normal('R', mu=y_hat["R"], sigma=sd, observed=data.iloc[:,2])


In [6]:
with sir_m:
    trace = pm.sample(2000, tune=2000, cores=2, chains=2)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


KeyboardInterrupt: 

In [ ]:
az.plot_trace(trace)

In [ ]:
pd.DataFrame([
    sir_m['I'].eval(),
    sir_m['R'].eval()
],
    index=['I', 'R']
).T.plot()